# MOPACを実行することもできる。

In [1]:
!"C:\Program Files\MOPAC\MOPAC2016.exe" C:\winmos10\UserData\phenol.dat

In [2]:
import glob

In [3]:
glob.glob("phenol\*.dat")

['phenol\\phenol.dat', 'phenol\\phenol2.dat']

In [4]:
for dat in glob.glob("phenol\*.dat"):
    !"C:\program files\mopac\MOPAC2016.exe" $dat

# 計算ファイルの作成

In [5]:
from rdkit import Chem
from rdkit.Chem import AllChem, PandasTools, MolStandardize
from tqdm.notebook import tqdm

In [6]:
folder_name = "mopac/"

In [7]:
df = PandasTools.LoadSDF('PubChem_compound_list_antioxidant.sdf')

In [8]:
mols = df["ROMol"]

In [9]:
mols[:10]

0    <img data-content="rdkit/molecule" src="data:i...
1    <img data-content="rdkit/molecule" src="data:i...
2    <img data-content="rdkit/molecule" src="data:i...
3    <img data-content="rdkit/molecule" src="data:i...
4    <img data-content="rdkit/molecule" src="data:i...
5    <img data-content="rdkit/molecule" src="data:i...
6    <img data-content="rdkit/molecule" src="data:i...
7    <img data-content="rdkit/molecule" src="data:i...
8    <img data-content="rdkit/molecule" src="data:i...
9    <img data-content="rdkit/molecule" src="data:i...
Name: ROMol, dtype: object

In [10]:
# RDKitによるコンフォマーの生成を参考に作成
# https://future-chem.com/rdkit-conformer/

errorlist=[]
lfc = MolStandardize.fragment.LargestFragmentChooser()

for mol in tqdm(mols[:10]):
    try:
        #一番大きなフラグメントを選ぶ
        mol = lfc.choose(mol)
        #molファイルをsmilesにする
        smiles=Chem.MolToSmiles(mol)
        #初期化
        energy=[]
        #水素の付加
        mol = Chem.AddHs(mol)
        #3次元構造の生成
        AllChem.EmbedMolecule(mol)
        #
        cids = AllChem.EmbedMultipleConfs(mol, numConfs=1000, randomSeed=1234,
                                           pruneRmsThresh=2, numThreads=0)
        for cid in cids:
            prop = AllChem.MMFFGetMoleculeProperties(mol)
            mmff = AllChem.MMFFGetMoleculeForceField(mol, prop, confId=cid)
            mmff.Minimize()
            
            energy.append((mmff.CalcEnergy(), cid))

            energy.sort()
            conf = mol.GetConformer(energy[0][1])

            #ファイルを作成
            with open(folder_name + smiles+'.dat', 'w') as f: 
                print('PM7 EF PRECISE GNORM=0.05 NOINTER GRAPHF VECTORS MMOK', file=f)
                print(smiles, file=f)
                print('Good Luck!', file=f)
                for n,(x,y,z) in enumerate(conf.GetPositions()):
                    atom = mol.GetAtomWithIdx(n)
                    print('{}  {:.6f} 1 {:.6f} 1 {:.6f} 1'.format(atom.GetSymbol(),x,y,z), file=f)
           
    except:
        print(smiles)
        errorlist.append(smiles)

  0%|          | 0/10 [00:00<?, ?it/s]

# 作成したファイルを実行する。

In [11]:
calc_dats = glob.glob(folder_name +"*.dat")

In [12]:
len(calc_dats)

10

In [13]:
for dat in tqdm(calc_dats):
    !"C:\program files\mopac\MOPAC2016.exe" $dat

  0%|          | 0/10 [00:00<?, ?it/s]

# データを読み取る

In [14]:
import os
import glob
import pandas as pd

In [15]:
files = glob.glob("mopac\*.arc")
files

['mopac\\COc1ccc2[nH]cc(CCNC(C)=O)c2c1.arc',
 'mopac\\CS(C)=O.arc',
 'mopac\\N=O.arc',
 'mopac\\Nc1c(O)cccc1C(=O)O.arc',
 'mopac\\O=C(O)C(=O)Cc1c[nH]c2ccccc12.arc',
 'mopac\\O=C(O)CCCCC(S)CCS.arc',
 'mopac\\O=C(O)CCCCC1CCSS1.arc',
 'mopac\\O=c1[nH]c(=O)c2[nH]c(=O)[nH]c2[nH]1.arc',
 'mopac\\Oc1ccc(O)cc1.arc',
 'mopac\\Oc1cccc(O)c1O.arc']

In [16]:
names = []
List=[]

for file in files:
    name = os.path.splitext(os.path.basename(file))[0]
    names.append(name)
    print(name)
    with open(file) as f:
        # 読み込んだファイルをリストに入れている。
        lines = f.readlines()
        #stripメソッドで空白や特定文字の削除
        lines_strip = [line.strip() for line in lines]
        #特定の文字列があるかを判定
        HOMO_line = [line for line in lines_strip if 'HOMO' in line]
        #空でなかったら、特定の場所にある数字を加えている。
        if HOMO_line != []:
            HOMO=float(str(HOMO_line[0][30:39]))
            print(HOMO)
        else:
            print("error")
        
    List.append(HOMO)

COc1ccc2[nH]cc(CCNC(C)=O)c2c1
-8.09
CS(C)=O
-8.62
N=O
-9.9
Nc1c(O)cccc1C(=O)O
-8.48
O=C(O)C(=O)Cc1c[nH]c2ccccc12
-8.7
O=C(O)CCCCC(S)CCS
-9.01
O=C(O)CCCCC1CCSS1
-8.04
O=c1[nH]c(=O)c2[nH]c(=O)[nH]c2[nH]1
-9.21
Oc1ccc(O)cc1
-8.78
Oc1cccc(O)c1O
-9.25


In [17]:
names

['COc1ccc2[nH]cc(CCNC(C)=O)c2c1',
 'CS(C)=O',
 'N=O',
 'Nc1c(O)cccc1C(=O)O',
 'O=C(O)C(=O)Cc1c[nH]c2ccccc12',
 'O=C(O)CCCCC(S)CCS',
 'O=C(O)CCCCC1CCSS1',
 'O=c1[nH]c(=O)c2[nH]c(=O)[nH]c2[nH]1',
 'Oc1ccc(O)cc1',
 'Oc1cccc(O)c1O']

In [18]:
List

[-8.09, -8.62, -9.9, -8.48, -8.7, -9.01, -8.04, -9.21, -8.78, -9.25]

In [19]:
df_read = pd.DataFrame(List, index = names, columns=["homo"])
df_read

,homo
COc1ccc2[nH]cc(CCNC(C)=O)c2c1,-8.09
CS(C)=O,-8.62
N=O,-9.90
Nc1c(O)cccc1C(=O)O,-8.48
O=C(O)C(=O)Cc1c[nH]c2ccccc12,-8.70
O=C(O)CCCCC(S)CCS,-9.01
O=C(O)CCCCC1CCSS1,-8.04
O=c1[nH]c(=O)c2[nH]c(=O)[nH]c2[nH]1,-9.21
Oc1ccc(O)cc1,-8.78
Oc1cccc(O)c1O,-9.25


In [21]:
df.to_csv('mopac_calc_antioxidant10.csv')

# コメント

1. 並列計算などができるようになったほうが､効率的
2. 途中で止められるような工夫
3. ファイル整理の工夫
4. エラーヘの対処
5. 配座探索の方法は他にも色々ある。